# PREDICTIVE MODELLING


## Applying logistic regression to predict the users whose gender type is unknown

### Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/eda_parent_dataset.csv')

In [ ]:
df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Hour,Age,Age_group
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31929.0,Subscriber,1992.0,1.0,2.0,30.0,20-29
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31845.0,Subscriber,1969.0,2.0,12.0,53.0,50-59
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31708.0,Subscriber,1946.0,1.0,12.0,76.0,60+
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.033459,31697.0,Subscriber,1994.0,1.0,14.0,28.0,20-29
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.033459,31861.0,Subscriber,1991.0,1.0,17.0,31.0,30-39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341053,2071,2018-12-25 19:43:31.611,2018-12-25 20:18:02.623,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,29225.0,Subscriber,1990.0,1.0,19.0,32.0,30-39
341054,587,2018-12-25 21:33:45.390,2018-12-25 21:43:32.415,3199,Newport Pkwy,40.728745,-74.032108,3481,York St,40.716490,-74.032108,29225.0,Subscriber,1988.0,2.0,21.0,34.0,30-39
341055,431,2018-12-26 00:01:42.949,2018-12-26 00:08:54.253,3199,Newport Pkwy,40.728745,-74.032108,3187,Warren St,40.721124,-74.032108,29641.0,Subscriber,1972.0,1.0,0.0,50.0,40-49
341056,305,2018-12-26 09:36:06.417,2018-12-26 09:41:11.967,3199,Newport Pkwy,40.728745,-74.032108,3187,Warren St,40.721124,-74.032108,29120.0,Subscriber,1980.0,1.0,9.0,42.0,40-49


### Training data - extracting users whose gender is already given

In [ ]:
train_data = df.query('gender==1.0 or gender==2.0') # 1.0 refers to male and 2.0 refers to female

In [ ]:
print("No. of users whose gender is already known", len(train_data))
print("No. of users who are females: ", len(train_data.query('gender==2.0')))
print("No. of users who are males: ", len(train_data.query('gender==1.0'))) 

No. of users whose gender is already known 320080
No. of users who are females:  72382
No. of users who are males:  247698


### Making the number of rows of males and females in training data similar

In [ ]:
train_data = train_data.sort_values('gender')
train_data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Hour,Age,Age_group
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31929.0,Subscriber,1992.0,1.0,2.0,30.0,20-29
214288,451,2018-08-27 18:39:45.483,2018-08-27 18:47:16.874,3220,5 Corners Library,40.734961,-74.059503,3210,Pershing Field,40.742677,-74.059503,29303.0,Subscriber,1990.0,1.0,18.0,32.0,30-39
214291,358,2018-08-28 02:58:39.201,2018-08-28 03:04:37.989,3220,5 Corners Library,40.734961,-74.059503,3210,Pershing Field,40.742677,-74.059503,26201.0,Subscriber,1969.0,1.0,2.0,53.0,50-59
214292,719,2018-08-28 08:02:40.949,2018-08-28 08:14:40.552,3220,5 Corners Library,40.734961,-74.059503,3638,Washington St,40.724294,-74.059503,29681.0,Subscriber,1982.0,1.0,8.0,40.0,30-39
214293,762,2018-08-28 10:20:49.690,2018-08-28 10:33:32.196,3220,5 Corners Library,40.734961,-74.059503,3275,Columbus Drive,40.718355,-74.059503,29657.0,Subscriber,1971.0,1.0,10.0,51.0,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271158,677,2018-10-21 15:23:11.048,2018-10-21 15:34:28.486,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.033459,33621.0,Subscriber,1985.0,2.0,15.0,37.0,30-39
113980,583,2018-06-21 08:38:22.173,2018-06-21 08:48:05.336,3193,Lincoln Park,40.724605,-74.078406,3195,Sip Ave,40.730743,-74.078406,29305.0,Subscriber,1985.0,2.0,8.0,37.0,30-39
271155,396,2018-10-21 14:42:39.114,2018-10-21 14:49:15.604,3183,Exchange Place,40.716247,-74.033459,3187,Warren St,40.721124,-74.033459,26298.0,Subscriber,1982.0,2.0,14.0,40.0,30-39
271168,316,2018-10-21 16:52:25.734,2018-10-21 16:57:42.105,3183,Exchange Place,40.716247,-74.033459,3481,York St,40.716490,-74.033459,33668.0,Subscriber,1968.0,2.0,16.0,54.0,50-59


In [ ]:
train_data.drop(train_data.head(180858).index, inplace=True) # dropping some male users since female users are very less
train_data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Hour,Age,Age_group
18881,528,2018-02-21 14:45:16.961,2018-02-21 14:54:05.090,3199,Newport Pkwy,40.728745,-74.032108,3276,Marin Light Rail,40.714584,-74.032108,29307.0,Subscriber,1975.0,1.0,14.0,47.0,40-49
18884,408,2018-02-21 18:04:58.678,2018-02-21 18:11:47.006,3199,Newport Pkwy,40.728745,-74.032108,3203,Hamilton Park,40.727596,-74.032108,31808.0,Subscriber,1954.0,1.0,18.0,68.0,60+
18885,397,2018-02-21 18:38:08.367,2018-02-21 18:44:45.540,3199,Newport Pkwy,40.728745,-74.032108,3203,Hamilton Park,40.727596,-74.032108,31776.0,Subscriber,1983.0,1.0,18.0,39.0,30-39
18887,558,2018-02-21 19:25:41.966,2018-02-21 19:35:00.856,3199,Newport Pkwy,40.728745,-74.032108,3184,Paulus Hook,40.714145,-74.032108,29662.0,Subscriber,1963.0,1.0,19.0,59.0,50-59
18867,878,2018-02-21 08:20:52.916,2018-02-21 08:35:31.789,3199,Newport Pkwy,40.728745,-74.032108,3206,Hilltop,40.731169,-74.032108,31842.0,Subscriber,1960.0,1.0,8.0,62.0,60+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271158,677,2018-10-21 15:23:11.048,2018-10-21 15:34:28.486,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.033459,33621.0,Subscriber,1985.0,2.0,15.0,37.0,30-39
113980,583,2018-06-21 08:38:22.173,2018-06-21 08:48:05.336,3193,Lincoln Park,40.724605,-74.078406,3195,Sip Ave,40.730743,-74.078406,29305.0,Subscriber,1985.0,2.0,8.0,37.0,30-39
271155,396,2018-10-21 14:42:39.114,2018-10-21 14:49:15.604,3183,Exchange Place,40.716247,-74.033459,3187,Warren St,40.721124,-74.033459,26298.0,Subscriber,1982.0,2.0,14.0,40.0,30-39
271168,316,2018-10-21 16:52:25.734,2018-10-21 16:57:42.105,3183,Exchange Place,40.716247,-74.033459,3481,York St,40.716490,-74.033459,33668.0,Subscriber,1968.0,2.0,16.0,54.0,50-59


In [ ]:
print(len(train_data))
print(len(train_data.query('gender==2.0'))) # female
print(len(train_data.query('gender==1.0'))) # male

139222
72382
66840


### Preparing test data - users whose gender is unknown

In [ ]:
test_data = df.query('gender==0.0')

In [ ]:
len(test_data)

20977

### Logistic regression model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
y = train_data['gender']

In [ ]:
train_data

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Hour,Age,Age_group
18881,528,2018-02-21 14:45:16.961,2018-02-21 14:54:05.090,3199,Newport Pkwy,40.728745,-74.032108,3276,Marin Light Rail,40.714584,-74.032108,29307.0,Subscriber,1975.0,1.0,14.0,47.0,40-49
18884,408,2018-02-21 18:04:58.678,2018-02-21 18:11:47.006,3199,Newport Pkwy,40.728745,-74.032108,3203,Hamilton Park,40.727596,-74.032108,31808.0,Subscriber,1954.0,1.0,18.0,68.0,60+
18885,397,2018-02-21 18:38:08.367,2018-02-21 18:44:45.540,3199,Newport Pkwy,40.728745,-74.032108,3203,Hamilton Park,40.727596,-74.032108,31776.0,Subscriber,1983.0,1.0,18.0,39.0,30-39
18887,558,2018-02-21 19:25:41.966,2018-02-21 19:35:00.856,3199,Newport Pkwy,40.728745,-74.032108,3184,Paulus Hook,40.714145,-74.032108,29662.0,Subscriber,1963.0,1.0,19.0,59.0,50-59
18867,878,2018-02-21 08:20:52.916,2018-02-21 08:35:31.789,3199,Newport Pkwy,40.728745,-74.032108,3206,Hilltop,40.731169,-74.032108,31842.0,Subscriber,1960.0,1.0,8.0,62.0,60+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271158,677,2018-10-21 15:23:11.048,2018-10-21 15:34:28.486,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.033459,33621.0,Subscriber,1985.0,2.0,15.0,37.0,30-39
113980,583,2018-06-21 08:38:22.173,2018-06-21 08:48:05.336,3193,Lincoln Park,40.724605,-74.078406,3195,Sip Ave,40.730743,-74.078406,29305.0,Subscriber,1985.0,2.0,8.0,37.0,30-39
271155,396,2018-10-21 14:42:39.114,2018-10-21 14:49:15.604,3183,Exchange Place,40.716247,-74.033459,3187,Warren St,40.721124,-74.033459,26298.0,Subscriber,1982.0,2.0,14.0,40.0,30-39
271168,316,2018-10-21 16:52:25.734,2018-10-21 16:57:42.105,3183,Exchange Place,40.716247,-74.033459,3481,York St,40.716490,-74.033459,33668.0,Subscriber,1968.0,2.0,16.0,54.0,50-59


In [ ]:
x= train_data[['tripduration', 'start station id', 'end station id', 'bikeid' , 'Age', 'Hour' ]] # using only those features that will be useful in prediction

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
xtrain = sc_x.fit_transform(xtrain) 
xtest = sc_x.transform(xtest)
  
print (xtrain[0:10, :])

[[-0.04641827 -0.29489753 -0.43331217  0.83590602  1.00244312 -2.26526214]
 [-0.06396056 -0.39897034 -0.37450043  1.61126896  0.11394332 -0.12821263]
 [-0.06986766 -0.58481463  0.11804788  0.02145586  0.21266552  1.23172797]
 [-0.07326872  3.0949024  -0.52152978 -1.31109949 -0.67583427 -1.09959877]
 [-0.03460407  0.03218843 -0.50682684  0.00749613 -0.67583427  0.64889629]
 [-0.02959199  0.07679106 -0.31568869 -0.14247119 -1.16944527 -0.12821263]
 [ 0.00388157 -0.45844051  0.15480522 -1.32386266 -1.07072307  0.84317352]
 [-0.01294471  0.03218843 -0.50682684  0.02305125 -0.57711207  0.0660646 ]
 [-0.06127552 -0.58481463  0.17685962 -1.31787992 -0.08350108  0.64889629]
 [-0.06467658  0.03218843 -0.37450043 -0.01523828  0.60755432  0.64889629]]


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(xtrain, ytrain)

LogisticRegression(random_state=0)

In [ ]:
y_pred = classifier.predict(xtest) # predicting for test split of data to find out the accuracy of the model
y_pred

array([1., 1., 2., ..., 1., 2., 1.])

### Evaluating the performance of the model

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, y_pred)
  
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[ 7198  9454]
 [ 5618 12536]]


In [ ]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(ytest, y_pred))

Accuracy :  0.5669712118600241


In [ ]:
count=0
female=0
for i in range(len(y_pred)):
  if(y_pred[i]==1):
    count+=1
  if(y_pred[i]==2):
    female+=1

print(count)
print(female)

12816
21990


### Performing the prediction for users whose gender is 0

In [ ]:
x_new = test_data[['tripduration', 'start station id', 'end station id', 'bikeid' , 'Age', 'Hour' ]]
x_new_numpy = x_new.to_numpy()
y_new = classifier.predict(x_new_numpy)
y_new=y_new.tolist()
x_new['gender']=y_new


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Analyzing the predicted values

In [ ]:
print("Total rows to be predicted : ", len(y_new))
countmale=0
countfemale=0
for i in range(len(y_new)):
  if(y_new[i]==1.0):
    countmale+=1
  if(y_new[i]==2.0):
    countfemale+=1

print("No. of users predicted to be male: ",countmale)
print("No. of users predicted to be female: ", countfemale)

Total rows to be predicted :  20977
No. of users predicted to be male:  20954
No. of users predicted to be female:  23


### Modifying the original dataframe to include the predicted gender values

In [ ]:
for index in df.index:
  if df['gender'][index]==0.0:
    df['gender'][index]=x_new['gender'][index]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Cleaning the modified dataset

In [ ]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Hour,Age,Age_group
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31929.0,Subscriber,1992.0,1.0,2.0,30.0,20-29
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31845.0,Subscriber,1969.0,2.0,12.0,53.0,50-59
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31708.0,Subscriber,1946.0,1.0,12.0,76.0,60+
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.033459,31697.0,Subscriber,1994.0,1.0,14.0,28.0,20-29
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.033459,31861.0,Subscriber,1991.0,1.0,17.0,31.0,30-39


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341058 entries, 0 to 341057
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             341058 non-null  int64  
 1   starttime                341058 non-null  object 
 2   stoptime                 341058 non-null  object 
 3   start station id         341058 non-null  int64  
 4   start station name       341058 non-null  object 
 5   start station latitude   341058 non-null  float64
 6   start station longitude  341058 non-null  float64
 7   end station id           341058 non-null  int64  
 8   end station name         341058 non-null  object 
 9   end station latitude     341058 non-null  float64
 10  end station longitude    341057 non-null  float64
 11  bikeid                   341057 non-null  float64
 12  usertype                 341057 non-null  object 
 13  birth year               341057 non-null  float64
 14  gend

In [ ]:
df = df.drop(341057) # dropping row with nan

In [ ]:
df = df.astype({"birth year":'int', "gender":'int', "Hour":'int', "Age":'int'})

In [ ]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Hour,Age,Age_group
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31929.0,Subscriber,1992,1,2,30,20-29
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31845.0,Subscriber,1969,2,12,53,50-59
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.033459,31708.0,Subscriber,1946,1,12,76,60+
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.033459,31697.0,Subscriber,1994,1,14,28,20-29
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.033459,31861.0,Subscriber,1991,1,17,31,30-39


In [ ]:
df["gender"].unique()

array([1, 2])

In [ ]:
df["starttime"] = pd.to_datetime(df["starttime"])
df["stoptime"] = pd.to_datetime(df["stoptime"])

### Saving the new dataset in a csv file

In [ ]:
df.to_csv('gender_predicted_dataset.csv', index=False)